In [ ]:
%pylab inline
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
sns.set_style('whitegrid')

# Загружаем данные

![](https://st.kp.yandex.net/images/logoOrange.png)

Будем учиться делить рецензии к фильмам на положительные и отрицательные.

In [ ]:
data = pd.read_csv('train.csv')
data.head(5)

In [ ]:
data.shape

In [ ]:
data.target.value_counts()

# Извлекаем признаки

Обычно методы машинного обучения работают с матрицей объект-признак, поэтому надо привести тексты рецензий к  этому виду. Есть несколько способов сделать это, мы будем использовать модель Bag of words.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Обучите CountVectorizer на текстах из выборки

In [ ]:
# Положите в X результат преобразования текстов с помощью CountVectorizer

Посмотрим на полученную матрицу `X`

In [ ]:
row_names = vec.get_feature_names()[:15]
col_names = [d[:30] for d in data.text[:15]]    
df = pd.DataFrame(X[:15, :15].todense(), index=col_names, columns=row_names)
sns.heatmap(df, annot=True);

Число в строке i и столбце j означает, сколько раз слово номер j встретилось в документе i. Слово '10' встречается во многих документах, потому что многие пишут "Ставлю оценку X из 10"

In [ ]:
X.shape

# Учим модель

Для примера будем использовать логистическую регрессию.
Это линейная модель, которая предсказывает итоговый класс по формуле $$y_{pred}(x) = sign(wx + b)$$ где `w` - веса признаков, `b` - смещение (в общем случае нужно, когда классы несбалансированны)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data.target, test_size=0.3, random_state=123)

# Обучите логистическую регрессию на X_train и положите в y_pred предсказание модели на X_test

## Посмотрим на коэффициенты

In [ ]:
# Выведите топ-50 слов с максимальными весами

In [ ]:
# Выведите топ-50 слов с минимальными весами

# Оцениваем качество

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_test, y_pred)

Дальше будем оптимизировать F1-score
$$F1 = \frac{2 \times precision \times recall}{precision + recall}$$

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
# Предскажите вероятности классов для строки 'Ужасный фильм, не рекомендую смотреть его'
# Объясните результат

## Кросс-валидация

In [ ]:
from sklearn.model_selection import cross_val_score

# Оцените f1-score построенной модели при помощи 5-fold cv
# Hint: рекомендую указывать n_jobs=-1

# Регуляризация

In [ ]:
from sklearn.model_selection import GridSearchCV

# Подберите значение параметра C для логистической регрессии, максимизирующее f1-score для логистичекой регрессии с l2- и l1-регуляризацией
# Hint: У GridSearchCV тоже можно указать n_jobs

In [ ]:
# Выведите количество ненулевых коэфициентов для лучших моделей с l1- и l2-регуляризацией

# N-граммы

In [ ]:
from sklearn.pipeline import make_pipeline

# Добавьте к токенам биграммы, постройте модель на этих признаках и оцените её качество.
# Hint: с помощью make_pipeline можно объединить векторизатор и классификатор в один объект, который удобно использовать для кроссвали дации и подбора параметров.

In [ ]:
# Выведите размер словаря и количество ненулевых коэфициентов

In [ ]:
# Выведите топ-50 токенов с максимальными и минимальными весами

# Нормализация

In [ ]:
import functools
import re
import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize


def memoized_with_single_argument(cache):
    """
    Copy-paste from https://github.com/kmike/pymorphy2/blob/master/pymorphy2/cache.py
    Basic caching decorator. It assumes a function only accepts
    a single argument, which is used as a cache key.

    >>> cache = {}
    >>> @memoized_with_single_argument(cache)
    ... def func(x):
    ...     return x*2
    >>> func(2)
    4
    >>> cache
    {2: 4}
    >>> cache[2] = 6
    >>> func(2)
    6
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(arg):
            if arg in cache:
                return cache[arg]
            res = func(arg)
            cache[arg] = res
            return res
        return wrapper
    return decorator


morph = pymorphy2.MorphAnalyzer()


@memoized_with_single_argument({})
def lemmatize(tok):
    # Исправьте эту функцию, чтобы возвращалась нормальная форма токена
    return tok


def tokenize2(text):
    return [tok for tok in simple_word_tokenize(text) if tok not in ',.']


def lemmatize_all(text):
    return [lemmatize(tok) for tok in tokenize2(text)]


In [ ]:
# 1. Обучите модель, использующую нормализацию и биграммы.
# Hint: функцию lemmatize_all можно подключить как "токенизатор" в CountVectorizer
# 2. Подберите оптимальные гиперпараметры, оцените качество.
# 3. Выведите 50 токенов с максимальними и минимальными весами.

# Влияние размера тренировочного набора на качество

In [ ]:
# copy-paste from scikit-learn docs

import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.model_selection import learning_curve


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and traning learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : integer, cross-validation generator, optional
        If an integer is passed, it is the number of folds (defaults to 3).
        Specific cross-validation objects can be passed, see
        sklearn.cross_validation module for the list of possible objects

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    print(test_scores_mean)
    return plt

In [ ]:
# Постройте графики зависимости качества модели от размера выборки при различных значениях параматра регуляризации C

# Задание

Получить f1-score больше 0.93.  
Я буду тестировать на новых текстах, поэтому надо сохранить в файл предсказания для text.csv.
### Идеи
- Лучше подобрать гиперпараметры
- Попробовать другие модели
- Придумать дополнительные признаки
- Скомбинировать несколько моделей